### SQLite Database in python

when we use the sqlite in pyhton we start from some steps.

step 1: import sqlite.

In [1]:
import sqlite3

step 2:connect to database.

when we use the sqlite database in python then we use the connection method.

In [2]:
conn=sqlite3.connect('sqlite.db')

step 3: create a table.

We create a cursor object to interact wiht the database and use the execute() mehtod to create a table with desired columns.

In [3]:
cursor =conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS employees
                  (id INTEGER PRIMARY KEY, name TEXT, position TEXT, salary REAL)""")
conn.commit()

step 4: insert data into the table.

we can insert data into the table using the execute() mehtod with placeholders:

In [4]:
cursor.execute("INSERT INTO employees (name, position, salary) VALUES (?, ?, ?)", ('John Doe', 'Software Engineer', 80000))
conn.commit()

Step 5: Query the data from the table.

To query data from the table, we use the execute() method to run a SELECT statement and the fetchall() method to retrieve the results:

In [5]:
cursor.execute("SELECT * FROM employees")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'John Doe', 'Software Engineer', 80000.0)


step 6: Update the data in the table.

To update data in the table, we use the execute() method with an UPDATE statement:

In [6]:
cursor.execute("UPDATE employees SET salary = ? WHERE id = ?", (90000, 1))
conn.commit()

Step 7: Delete data from the table.

To delete data from the table, we use the execute() method with a DELETE statement:

In [7]:
cursor.execute("DELETE FROM employees WHERE id = ?", (1,))
conn.commit()

Step 8: Close the connection.

It’s essential to close the connection when you’re done interacting with the database:

In [8]:
conn.close()

In [9]:
# sample_world.py
import sqlite3
from sqlite3 import IntegrityError

DB = "sample_world.db"

def get_conn(db=DB):
    # Use a row factory so results are accessible by column name: row["name"]
    conn = sqlite3.connect(db)
    conn.row_factory = sqlite3.Row
    # enable foreign keys if you plan to use them
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

def init_db(conn):
    """Create the contacts table."""
    with conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS contacts (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                email TEXT UNIQUE,
                phone TEXT,
                created_at TEXT DEFAULT (datetime('now'))
            );
        """)

def insert_contacts(conn, contacts):
    """
    contacts: iterable of (name, email, phone)
    uses executemany for bulk insert
    """
    with conn:
        conn.executemany(
            "INSERT INTO contacts (name, email, phone) VALUES (?, ?, ?);",
            contacts
        )

def list_contacts(conn):
    cur = conn.execute("SELECT * FROM contacts ORDER BY id;")
    rows = cur.fetchall()
    print("--- all contacts ---")
    for r in rows:
        # convert sqlite3.Row to dict for pretty print
        print(dict(r))
    print("---------------------")

def find_by_email(conn, email):
    cur = conn.execute("SELECT * FROM contacts WHERE email = ?;", (email,))
    return cur.fetchone()  # None or sqlite3.Row

def update_phone(conn, contact_id, new_phone):
    with conn:
        conn.execute("UPDATE contacts SET phone = ? WHERE id = ?;", (new_phone, contact_id))

def delete_contact(conn, contact_id):
    with conn:
        conn.execute("DELETE FROM contacts WHERE id = ?;", (contact_id,))

def transaction_example(conn):
    """
    Example transaction: swap two emails (this will rollback on integrity errors).
    Demonstrates automatic commit/rollback when using `with conn:`
    """
    try:
        with conn:  # enters a transaction, commits on success, rollbacks on exception
            a = conn.execute("SELECT id, email FROM contacts WHERE id = ?;", (1,)).fetchone()
            b = conn.execute("SELECT id, email FROM contacts WHERE id = ?;", (2,)).fetchone()
            if not a or not b:
                raise ValueError("need two contacts with id 1 and 2")
            # swap emails
            conn.execute("UPDATE contacts SET email = ? WHERE id = ?;", (b["email"], a["id"]))
            conn.execute("UPDATE contacts SET email = ? WHERE id = ?;", (a["email"], b["id"]))
            # if one of the emails violates UNIQUE, sqlite will raise IntegrityError and entire transaction rolls back
    except IntegrityError as e:
        print("Transaction failed, rolled back:", e)

def main():
    conn = get_conn()
    init_db(conn)

    # Insert sample data (ignore if already present)
    sample = [
        ("Alice", "alice@example.com", "555-0100"),
        ("Bob", "bob@example.com", "555-0110"),
        ("Charlie", "charlie@example.com", None)
    ]
    try:
        insert_contacts(conn, sample)
    except IntegrityError:
        # likely already inserted earlier — safe to ignore for this demo
        pass

    list_contacts(conn)

    # Find one by email
    r = find_by_email(conn, "bob@example.com")
    print("Found Bob:", dict(r) if r else "not found")

    # Update phone for id=2 (Bob)
    update_phone(conn, 2, "555-9999")
    print("After phone update:")
    list_contacts(conn)

    # Swap emails (shows transaction)
    transaction_example(conn)
    print("After transaction attempt:")
    list_contacts(conn)

    # Delete a contact
    delete_contact(conn, 3)  # delete Charlie
    print("After delete:")
    list_contacts(conn)

    conn.close()

if __name__ == "__main__":
    main()


--- all contacts ---
{'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'phone': '555-0100', 'created_at': '2025-08-08 11:46:05'}
{'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'phone': '555-0110', 'created_at': '2025-08-08 11:46:05'}
{'id': 3, 'name': 'Charlie', 'email': 'charlie@example.com', 'phone': None, 'created_at': '2025-08-08 11:46:05'}
---------------------
Found Bob: {'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'phone': '555-0110', 'created_at': '2025-08-08 11:46:05'}
After phone update:
--- all contacts ---
{'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'phone': '555-0100', 'created_at': '2025-08-08 11:46:05'}
{'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'phone': '555-9999', 'created_at': '2025-08-08 11:46:05'}
{'id': 3, 'name': 'Charlie', 'email': 'charlie@example.com', 'phone': None, 'created_at': '2025-08-08 11:46:05'}
---------------------
Transaction failed, rolled back: UNIQUE constraint failed: contacts.email
After transaction at

In [10]:
import sqlite3

conn = sqlite3.connect("library.db")
conn.row_factory = sqlite3.Row
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year INTEGER,
    available BOOLEAN DEFAULT 1
);
""")
conn.commit()
print("Table 'books' created (if not exists).")


Table 'books' created (if not exists).


In [11]:
cur.execute("INSERT INTO books (title, author, year) VALUES (?, ?, ?);",
            ("1984", "George Orwell", 1949))
conn.commit()
print("Inserted one book.")


Inserted one book.


In [12]:
books_to_add = [
    ("Brave New World", "Aldous Huxley", 1932),
    ("Fahrenheit 451", "Ray Bradbury", 1953),
    ("The Handmaid's Tale", "Margaret Atwood", 1985)
]
cur.executemany("INSERT INTO books (title, author, year) VALUES (?, ?, ?);", books_to_add)
conn.commit()
print("Inserted multiple books.")


Inserted multiple books.


In [13]:
cur.execute("SELECT * FROM books;")
all_books = cur.fetchall()
print("All books in library:")
for book in all_books:
    print(f"{book['id']}: {book['title']} by {book['author']} ({book['year']})")


All books in library:
1: 1984 by George Orwell (1949)
2: Brave New World by Aldous Huxley (1932)
3: Fahrenheit 451 by Ray Bradbury (1953)
4: The Handmaid's Tale by Margaret Atwood (1985)


In [14]:
cur.execute("SELECT * FROM books WHERE title = ?;", ("1984",))
book = cur.fetchone()
if book:
    print("Fetched one book with fetchone():")
    print(f"{book['id']}: {book['title']} by {book['author']} ({book['year']})")
else:
    print("Book not found.")


Fetched one book with fetchone():
1: 1984 by George Orwell (1949)


In [15]:
cur.execute("SELECT * FROM books;")
first_two = cur.fetchmany(2)
print("First two books with fetchmany():")
for book in first_two:
    print(f"{book['id']}: {book['title']}")


First two books with fetchmany():
1: 1984
2: Brave New World


In [16]:
cur.execute("UPDATE books SET available = 0 WHERE title = ?;", ("Fahrenheit 451",))
conn.commit()
print("Updated availability of 'Fahrenheit 451'.")


Updated availability of 'Fahrenheit 451'.


In [17]:
cur.execute("DELETE FROM books WHERE title = ?;", ("The Handmaid's Tale",))
conn.commit()
print("Deleted 'The Handmaid's Tale'.")


Deleted 'The Handmaid's Tale'.


In [18]:
cur.execute("SELECT * FROM books;")
books_left = cur.fetchall()
print("Books left after deletion:")
for book in books_left:
    avail = "Yes" if book["available"] else "No"
    print(f"{book['id']}: {book['title']} — Available? {avail}")


Books left after deletion:
1: 1984 — Available? Yes
2: Brave New World — Available? Yes
3: Fahrenheit 451 — Available? No


In [19]:
conn.close()
print("Connection closed.")


Connection closed.
